In [1]:
import ConfigArgs as args
import psycopg2
from tqdm import tqdm

In [2]:
conn = psycopg2.connect(args.conn_str)
cursor = conn.cursor()

In [3]:
sql = """SELECT image_name, triplets FROM "Image2GraphEmbedding_V2_MSCOCO";"""
cursor.execute(sql)
records = cursor.fetchall()

In [4]:
def parse_postgres_array(data):
    cleaned_data = data.replace('{', '').replace('}', '').replace('""', '"')
    list_data = cleaned_data.split('","')
    list_data = [item.strip('"') for item in list_data]
    
    return list_data
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    if not union:
        return 0.0  # Tránh chia cho 0
    return len(intersection) / len(union)
def extract_subject_relation_object(trips):
    trip = []
    for item in trips:
        words = item.split()
        t_json = {
            "subject": words[0],
            "relation": ' '.join(words[1:-1]),
            "object": words[-1]
        }
        trip.append(t_json)
    return trip

In [5]:
img_id = []
file_image = []
triplet_set = []
trip_by_im = []
for record in records[:]:
    file_image.append(record[0])
    triplet_set.append(set(parse_postgres_array(record[1])))

In [6]:
len(file_image)

83995

In [6]:
# combined = list(zip(triplet_set, file_image))
# similarities = [(filename, jaccard_similarity(setQ, triplet_set), triplet_set) for triplet_set, filename in combined]
# top_k_res = sorted(similarities, key=lambda x: x[1], reverse=True)[:2]
# top_k_res

In [9]:
res = []
combined = list(zip(triplet_set, file_image))
for item in tqdm(triplet_set[:]):
    setQ = item
    similarities = [(filename, jaccard_similarity(setQ, triplet_set), triplet_set) for triplet_set, filename in combined]
    top_k_res = sorted(similarities, key=lambda x: x[1], reverse=True)[:2]
    for idx, (i, s, t) in enumerate(top_k_res):
        if(idx == 1 and s >= 0.375):
            t_json = {
                "qe": {
                    "image_id": top_k_res[0][0],
                    "trip": list(top_k_res[0][2]),
                },
                "rev": {
                    "image_id": i,
                    "image_id_qe": top_k_res[0][0],
                    "trip": list(t) 
                    # "score": s,
                    # "trip": extract_subject_relation_object(list(t))
                }
            }
            res.append(t_json)
    # break
len(res)

  0%|          | 0/83995 [00:00<?, ?it/s]

100%|██████████| 83995/83995 [3:49:24<00:00,  6.10it/s]  


81845

In [ ]:
res[]

[{'qe': {'image_id': '000000322056.jpg',
   'trip': ['man riding horse',
    'hat on head',
    'horse has ear',
    'man wearing hat',
    'head of horse',
    'man wearing jacket',
    'ear of horse',
    'hat on man']},
  'rev': {'image_id': '000000072793.jpg',
   'image_id_qe': '000000322056.jpg',
   'trip': ['man riding horse',
    'hat on head',
    'horse has ear',
    'man wearing hat',
    'man on horse',
    'head of horse',
    'man wearing jacket',
    'ear of horse',
    'hat on man']}}]

In [10]:
import json
with open('Rev_v2.json', 'w') as f:
    json.dump(res, f)

In [1]:
import numpy as np
from sklearn.metrics import ndcg_score

queries = {
    "000000504037": {10: 10, 20: 20, 50: 50},
    "000000127540": {10: 10, 20: 20, 50: 50},
    "000000268185": {10: 7, 20: 11, 50: 30},
    "000000126282": {10: 9, 20: 16, 50: 38},
    "000000268518": {10: 10, 20: 19, 50: 47}
}

ndcg_results = {}
for query_id, scores in queries.items():
    ndcg_results[query_id] = {}
    
    for k, relevant_count in scores.items():

        y_true = np.array([1] * relevant_count + [0] * (k - relevant_count))

        y_score = np.linspace(1, 0, num=k)  # Điểm giảm dần từ 1 -> 0

        ndcg_results[query_id][f"NDCG@{k}"] = ndcg_score([y_true], [y_score])

for query, scores in ndcg_results.items():
    print(f"Query {query}: {scores}")



Query 000000504037: {'NDCG@10': 1.0, 'NDCG@20': 1.0, 'NDCG@50': 1.0000000000000002}
Query 000000127540: {'NDCG@10': 1.0, 'NDCG@20': 1.0, 'NDCG@50': 1.0000000000000002}
Query 000000268185: {'NDCG@10': 1.0, 'NDCG@20': 1.0, 'NDCG@50': 0.9999999999999998}
Query 000000126282: {'NDCG@10': 1.0, 'NDCG@20': 1.0, 'NDCG@50': 1.0}
Query 000000268518: {'NDCG@10': 1.0, 'NDCG@20': 1.0, 'NDCG@50': 1.0000000000000002}
